In [2]:
import os
import torch
import pandas as pd

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np 
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
#from torchsummary  import summary
import torch.nn as nn
import torch.optim as optim

In [10]:
class MyDataset(Dataset):

    def __init__(self, csv_path,label_transform =None,  transform=None):

        self.df = pd.read_csv(csv_path)
        
        #self.df = self.df.sample(n=100, random_state=42).reset_index(drop=True)  
        self.label_transform= label_transform
        self.transform = transform
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_path = self.df.loc[index, 'Images']
        #img = Image.open(image_path).convert("RGB")
        img = np.load(image_path)
        img = Image.fromarray(img.astype('uint8')).convert('RGB')
         
        img= np.array(img)
        
        label = self.df.loc[index,'365']
        
        
        ## clinical 
        sCD25 = self.df.loc[index,'sCD25(IL-2Ra)']
        BB14 = self.df.loc[index,'4-1BB']
        CTLA = self.df.loc[index,'CTLA-4']
        PDL1 = self.df.loc[index,'PD-L1'] 
        PD = self.df.loc[index,'PD-1']
        Tim = self.df.loc[index,'Tim-3']
        
        
        tabular = [[sCD25, BB14,CTLA , PDL1, PD, Tim]]
        tabular = torch.FloatTensor(tabular)

        
                            
        if self.transform is not None:
            img = self.transform(img)
        if self.label_transform is not None:
            label = self.label_transform(label)

        return img, tabular, label

    

In [11]:
train_transforms=transforms.Compose([transforms.ToPILImage(),
                              transforms.Resize((224,224)),
                             transforms.ToTensor(),
                             #transforms.RandomHorizontalFlip(),
                             #transforms.Normalize([0.2281, 0.2281, 0.2281], [0.2293, 0.2293, 0.2293])
                             ])
#transforms.RandomHorizontalFlip(),
#transforms.Normalize([0.4436, 0.4436, 0.4436], [0.2001, 0.2001, 0.2001]),

In [12]:
   train_dataset = MyDataset(
        csv_path='/root/code/thesis/codeFolder/LatestDataInUse/csv/CT2SequencesCombine_numpy_tumor.csv',
        transform=train_transforms)

In [13]:

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=32,
        drop_last=True,
        shuffle=True,  # want to shuffle the dataset
        num_workers=0)

In [14]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for images, clincial_data, labels in train_loader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(images, dim=[0,2,3])
        channels_squared_sum += torch.mean(images**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [ ]:
get_mean_and_std(train_loader)

In [15]:
#CT two sequences numpy tumor
(tensor([0.0381, 0.0381, 0.0381]), tensor([0.1041, 0.1041, 0.1041]))

(tensor([0.0381, 0.0381, 0.0381]), tensor([0.1041, 0.1041, 0.1041]))

In [9]:
#CT two sequences
(tensor([0.0498, 0.0498, 0.0498]), tensor([0.1235, 0.1235, 0.1235]))

In [7]:
#s3t1_wats_combine
tensor([0.1865, 0.1865, 0.1865]), tensor([0.2008, 0.2008, 0.2008]))

In [ ]:
#e_thrive-
(tensor([0.1469, 0.1469, 0.1469]), tensor([0.1824, 0.1824, 0.1824]))

In [8]:
#e-thrive_bh
(tensor([0.1634, 0.1634, 0.1634]), tensor([0.1898, 0.1898, 0.1898]))

In [ ]:
#t2_hr_spir_range
(tensor([0.0716, 0.0716, 0.0716]), tensor([0.1088, 0.1088, 0.1088]))

In [10]:
# MRT4Sequences
(tensor([0.1391, 0.1391, 0.1391]), tensor([0.1779, 0.1779, 0.1779]))

NameError: name 'tensor' is not defined

In [ ]:
# MRT5Sequences